In [1]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import tools

In [2]:
sitetype_file = "/Users/jpnousu/WBT_data/rasters/vmi_16/sitefertility_mnfi.asc" 
lai_eg_shrub_f = r'/Users/jpnousu/WBT_data/rasters/rasanen/lai_evergreen_shrubs.tif' 
lai_eg_m_shrub_f = r'/Users/jpnousu/WBT_data/rasters/rasanen/lai_evergreen_shrubs_match.tif' 
lai_de_shrub_f = r'/Users/jpnousu/WBT_data/rasters/rasanen/lai_deciduous_shrubs.tif' 
lai_de_m_shrub_f = r'/Users/jpnousu/WBT_data/rasters/rasanen/lai_deciduous_shrubs_match.tif' 
lai_gram_f = r'/Users/jpnousu/WBT_data/rasters/rasanen/lai_graminoids.tif' 
lai_gram_m_f = r'/Users/jpnousu/WBT_data/rasters/rasanen/lai_graminoids_match.asc' 

# --- Evergreen shrubs ---
with rasterio.open(lai_eg_shrub_f) as src:
    LAI_eg_shrub = src.read(1)
    eg_meta = src.meta.copy()
    old_nodata = src.nodata

if old_nodata is not None:
    LAI_eg_shrub = np.where(LAI_eg_shrub == old_nodata, -9999, LAI_eg_shrub)
LAI_eg_shrub[LAI_eg_shrub <= 0.] = -9999

eg_meta.update({"nodata": -9999, "dtype": "float32"})

intermediate_eg_file = r'/Users/jpnousu/WBT_data/rasters/rasanen/lai_evergreen_shrubs_clean.asc'
with rasterio.open(intermediate_eg_file, "w", **eg_meta) as dst:
    dst.write(LAI_eg_shrub, 1)

# --- Deciduous shrubs ---
with rasterio.open(lai_de_shrub_f) as src:
    LAI_de_shrub = src.read(1)
    de_meta = src.meta.copy()
    old_nodata = src.nodata

if old_nodata is not None:
    LAI_de_shrub = np.where(LAI_de_shrub == old_nodata, -9999, LAI_de_shrub)
LAI_de_shrub[LAI_de_shrub <= 0.] = -9999
de_meta.update({"nodata": -9999, "dtype": "float32"})

intermediate_de_file = r'/Users/jpnousu/WBT_data/rasters/rasanen/lai_deciduous_shrubs_clean.asc'
with rasterio.open(intermediate_de_file, "w", **de_meta) as dst:
    dst.write(LAI_de_shrub, 1)

# --- Graminoids ---
with rasterio.open(lai_gram_f) as src:
    LAI_gram = src.read(1)
    gram_meta = src.meta.copy()
    old_nodata = src.nodata

if old_nodata is not None:
    LAI_gram = np.where(LAI_gram == old_nodata, -9999, LAI_gram)
LAI_gram[LAI_gram <= 0.] = -9999

gram_meta.update({"nodata": -9999, "dtype": "float32"})

intermediate_gram_file = r'/Users/jpnousu/WBT_data/rasters/rasanen/lai_graminoids_clean.asc'
with rasterio.open(intermediate_gram_file, "w", **gram_meta) as dst:
    dst.write(LAI_gram, 1)

# let's make the files match the sitetype_file
tools.reproj_match(intermediate_gram_file, sitetype_file, lai_gram_m_f, resampling_method='average', save_in='asc')
tools.reproj_match(intermediate_eg_file, sitetype_file, lai_eg_m_shrub_f, resampling_method='average', save_in='asc')
tools.reproj_match(intermediate_de_file, sitetype_file, lai_de_m_shrub_f, resampling_method='average', save_in='asc')

Coregistered to shape: 1250 1250 
 Affine | 16.00, 0.00, 370000.00|
| 0.00,-16.00, 7557500.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 1250 1250 
 Affine | 16.00, 0.00, 370000.00|
| 0.00,-16.00, 7557500.00|
| 0.00, 0.00, 1.00|
Coregistered to shape: 1250 1250 
 Affine | 16.00, 0.00, 370000.00|
| 0.00,-16.00, 7557500.00|
| 0.00, 0.00, 1.00|


In [3]:
# --- Read matched LAI rasters ---
with rasterio.open(lai_eg_m_shrub_f) as src:
    LAI_eg_shrub = src.read(1)
    meta = src.meta.copy()

with rasterio.open(lai_de_m_shrub_f) as src:
    LAI_de_shrub = src.read(1)

with rasterio.open(lai_gram_m_f) as src:
    LAI_grass = src.read(1)

LAI_grass[LAI_grass == -9999] = np.nan
LAI_de_shrub[LAI_de_shrub == -9999] = np.nan
LAI_eg_shrub[LAI_eg_shrub == -9999] = np.nan

# --- Compute total shrub LAI ---
LAI_shrub = LAI_eg_shrub + LAI_de_shrub

# --- Read sitetype classification ---
with rasterio.open(sitetype_file) as src:
    sitetype = src.read(1)

# --- Fill missing values in LAI rasters using mean per sitetype ---
for unique_type in np.unique(sitetype):
    mask = (sitetype == unique_type)

    # Shrub LAI
    valid_vals = LAI_shrub[mask & ~np.isnan(LAI_shrub)]
    if valid_vals.size > 0:
        mean_val = np.nanmean(valid_vals)
        LAI_shrub[mask & np.isnan(LAI_shrub)] = mean_val
    else:
        # If no valid LAI values for this type, assign a small default
        LAI_shrub[mask & np.isnan(LAI_shrub)] = 0.01

    # Grass LAI
    valid_vals = LAI_grass[mask & ~np.isnan(LAI_grass)]
    if valid_vals.size > 0:
        mean_val = np.nanmean(valid_vals)
        LAI_grass[mask & np.isnan(LAI_grass)] = mean_val
    else:
        LAI_grass[mask & np.isnan(LAI_grass)] = 0.01

small_val = 0.001
LAI_grass[LAI_grass < small_val] = small_val
LAI_shrub[LAI_shrub < small_val] = small_val

# --- LAI Grass ---
out_LAI_grass_file = r'/Users/jpnousu/WBT_data/rasters/rasanen/LAI_grass.asc'
with rasterio.open(out_LAI_grass_file, "w", **meta) as dst:
    dst.write(LAI_grass, 1)

# --- LAI Shrub ---
out_LAI_shrub_file = r'/Users/jpnousu/WBT_data/rasters/rasanen/LAI_shrub.asc'
with rasterio.open(out_LAI_shrub_file, "w", **meta) as dst:
    dst.write(LAI_shrub, 1)